In [ ]:
#Performing intergration of CodeLlama
!pip install git+https://github.com/huggingface/transformers.git@main accelerate


  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-zt_oeob6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-zt_oeob6
  Resolved https://github.com/huggingface/transformers.git to commit 19dabe96362803fb0a9ae7073d03533966598b17
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_name = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def generate_code(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = "Write a Python function to calculate the factorial of a number."
generated_code = generate_code(prompt)
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Write a Python function to calculate the factorial of a number.

Examples
factorial(5) ➞ 120

factorial(4) ➞ 24

factorial(3) ➞ 6

factorial(2) ➞ 2

factorial(1) ➞ 1

factorial(0) ➞ 1




In [ ]:
def executor(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# Test the executor
test_prompt = "Write a Python function to calculate the factorial of a number."
result = executor(test_prompt)
print(result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Write a Python function to calculate the factorial of a number.

Examples
factorial(5) ➞ 120

factorial(4) ➞ 24

factorial(3) ➞ 6

factorial(2) ➞ 2

factorial(1) ➞ 1

factorial(0) ➞ 1

Notes
Factorials are calculated by multiplying all the numbers from 1 to n.

Factorials of 0 and 1 are both 1.

Factorials of negative numbers are undefined.

Factorials of floats are undefined.

Factorials of complex numbers are undefined.

Factorials of strings are undefined.

Factorials of lists are undefined.

Factor


In [ ]:
import ast

def is_valid_python(code_string):
    try:
        ast.parse(code_string)
        return True
    except SyntaxError:
        return False

# Test the validation
valid = is_valid_python(result)
print(f"Is the generated code valid Python? {valid}")

Is the generated code valid Python? False


In [ ]:
def planner(task):
    prompt = f"Decompose the following coding task into smaller subtasks:\n{task}\nProvide the subtasks as a numbered list."
    decomposition = executor(prompt)
    subtasks = [line.strip() for line in decomposition.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]
    return subtasks

# Test the planner
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
subtasks = planner(main_task)
print("Subtasks:")
for task in subtasks:
    print(task)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Subtasks:


In [ ]:
def recursive_code_generation(task, depth=0, max_depth=3):
    if depth >= max_depth:
        return "Max recursion depth reached. Task too complex."

    # Try to generate code
    generated_code = executor(task)

    # Validate the generated code
    if is_valid_python(generated_code):
        return generated_code

    # If invalid, decompose the task
    subtasks = planner(task)

    # Recursively generate code for subtasks
    subtask_results = []
    for subtask in subtasks:
        subtask_code = recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    # Combine subtask results
    combined_code = "\n\n".join(subtask_results)
    return combined_code

main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = recursive_code_generation(main_task)
print("Final Generated Code:")
print(final_code)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Final Generated Code:



In [ ]:
with open("generated_code.py", "w") as f:
    f.write(final_code)
print("Generated code has been saved to 'generated_code.py'")

Generated code has been saved to 'generated_code.py'


In [ ]:
print(tokenizer.eos_token, tokenizer.eos_token_id)

</s> 2


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import ast

# Load the model and tokenizer
model_name = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Set pad_token_id explicitly
tokenizer.pad_token_id = tokenizer.eos_token_id

def executor(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.pad_token_id)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

def is_valid_python(code_string):
    try:
        ast.parse(code_string)
        return True
    except SyntaxError:
        return False

def planner(task):
    prompt = f"Decompose the following coding task into smaller subtasks:\n{task}\nProvide the subtasks as a numbered list."
    decomposition = executor(prompt)
    subtasks = [line.strip() for line in decomposition.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]
    return subtasks

def recursive_code_generation(task, depth=0, max_depth=3):
    print(f"Depth {depth}: Processing task: {task}")

    if depth >= max_depth:
        return "Max recursion depth reached. Task too complex."

    # Try to generate code
    generated_code = executor(task)

    # Validate the generated code
    if is_valid_python(generated_code):
        print(f"Depth {depth}: Valid code generated")
        return generated_code

    print(f"Depth {depth}: Invalid code. Decomposing task")
    # If invalid, decompose the task
    subtasks = planner(task)

    # Recursively generate code for subtasks
    subtask_results = []
    for subtask in subtasks:
        subtask_code = recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    # Combine subtask results
    combined_code = "\n\n".join(subtask_results)
    return combined_code

# Test the recursive pipeline
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = recursive_code_generation(main_task)

print("\nFinal Generated Code:")
print("=" * 40)
print(final_code)
print("=" * 40)

# Save the generated code to a file
with open("generated_code.py", "w") as f:
    f.write(final_code)
print("Generated code has been saved to 'generated_code.py'")

# Test the executor separately
test_prompt = "Write a Python function to calculate the factorial of a number."
test_result = executor(test_prompt)
print("\nTest Executor Result:")
print("=" * 40)
print(test_result)
print("=" * 40)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Depth 0: Processing task: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.
Depth 0: Invalid code. Decomposing task

Final Generated Code:

Generated code has been saved to 'generated_code.py'

Test Executor Result:
Write a Python function to calculate the factorial of a number.

Examples
factorial(5) ➞ 120

factorial(4) ➞ 24

factorial(3) ➞ 6

factorial(2) ➞ 2

factorial(1) ➞ 1

factorial(0) ➞ 1

Notes
Factorials are calculated by multiplying all the numbers from 1 to n.

Factorials of 0 and 1 are both 1.

Factorials of negative numbers are undefined.

Factorials of floats are undefined.

Factorials of complex numbers are undefined.

Factorials of strings are undefined.

Factorials of lists are undefined.

Factor


In [ ]:
## OLD CODE. NOT WORKING WELL

from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
import ast

# Load the model and tokenizer
model_name = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Set pad_token_id explicitly
tokenizer.pad_token_id = tokenizer.eos_token_id

# Create a generation config
generation_config = GenerationConfig(
    max_length=200,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
)

def executor(prompt):
    print(f"Executing prompt: {prompt[:50]}...")  # Print the first 50 characters of the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    try:
        outputs = model.generate(**inputs, generation_config=generation_config)
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Generated code length: {len(generated_code)}")
        if len(generated_code) == 0:
            print("Warning: Generated code is empty")
        return generated_code
    except Exception as e:
        print(f"Error in code generation: {str(e)}")
        return ""

def is_valid_python(code_string):
    if not code_string.strip():
        print("Warning: Empty code string")
        return False
    try:
        ast.parse(code_string)
        return True
    except SyntaxError as e:
        print(f"Syntax error in generated code: {str(e)}")
        return False

def planner(task):
    prompt = f"Decompose the following coding task into smaller subtasks:\n{task}\nProvide the subtasks as a numbered list."
    decomposition = executor(prompt)
    subtasks = [line.strip() for line in decomposition.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]
    print(f"Planned subtasks: {subtasks}")
    return subtasks

def recursive_code_generation(task, depth=0, max_depth=3):
    print(f"Depth {depth}: Processing task: {task}")

    if depth >= max_depth:
        return "Max recursion depth reached. Task too complex."

    generated_code = executor(task)

    if is_valid_python(generated_code):
        print(f"Depth {depth}: Valid code generated")
        return generated_code

    print(f"Depth {depth}: Invalid code. Decomposing task")
    subtasks = planner(task)

    subtask_results = []
    for subtask in subtasks:
        subtask_code = recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    combined_code = "\n\n".join(subtask_results)
    return combined_code

# Test the recursive pipeline
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = recursive_code_generation(main_task)

print("\nFinal Generated Code:")
print("=" * 40)
print(final_code)
print("=" * 40)

# Save the generated code to a file
with open("generated_code.py", "w") as f:
    f.write(final_code)
print("Generated code has been saved to 'generated_code.py'")

# Test the executor separately
test_prompt = "Write a Python function to calculate the factorial of a number."
test_result = executor(test_prompt)
print("\nTest Executor Result:")
print("=" * 40)
print(test_result)
print("=" * 40)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, GenerationConfig
import torch
import ast
from sklearn.cluster import KMeans

# Load CodeLlama model and tokenizer
codellama_name = "codellama/CodeLlama-7b-hf"
codellama_tokenizer = AutoTokenizer.from_pretrained(codellama_name)
codellama_model = AutoModelForCausalLM.from_pretrained(codellama_name, torch_dtype=torch.float16, device_map="auto")

# Set pad_token_id explicitly for CodeLlama
codellama_tokenizer.pad_token_id = codellama_tokenizer.eos_token_id

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Create a generation config for CodeLlama
generation_config = GenerationConfig(
    max_length=200,
    pad_token_id=codellama_tokenizer.pad_token_id,
    eos_token_id=codellama_tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
)

def reframe_prompt(prompt):
    inputs = bert_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    # Use embeddings to generate a simplified prompt (placeholder logic)
    simplified_prompt = f"Simplify and implement: {prompt}"
    return simplified_prompt

def improved_planner(task):
    # Generate potential subtasks
    subtask_prompt = f"List subtasks for: {task}\nProvide the subtasks as a numbered list."
    subtask_text = executor(subtask_prompt)
    potential_subtasks = [line.strip() for line in subtask_text.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]

    # Use BERT embeddings to cluster and identify subtasks
    if len(potential_subtasks) > 1:
        inputs = bert_tokenizer(potential_subtasks, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)

        # Cluster embeddings
        n_clusters = min(len(potential_subtasks), 3)  # Limit to 3 clusters
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(embeddings)

        # Select representative subtask from each cluster
        subtasks = []
        for i in range(n_clusters):
            cluster_subtasks = [s for s, l in zip(potential_subtasks, kmeans.labels_) if l == i]
            if cluster_subtasks:
                subtasks.append(cluster_subtasks[0])
    else:
        subtasks = potential_subtasks

    return subtasks

def executor(prompt):
    print(f"Executing prompt: {prompt[:50]}...")  # Print the first 50 characters of the prompt
    inputs = codellama_tokenizer(prompt, return_tensors="pt").to(codellama_model.device)
    try:
        outputs = codellama_model.generate(**inputs, generation_config=generation_config)
        generated_code = codellama_tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Generated code length: {len(generated_code)}")
        if len(generated_code) == 0:
            print("Warning: Generated code is empty")
        return generated_code
    except Exception as e:
        print(f"Error in code generation: {str(e)}")
        return ""

def is_valid_python(code_string):
    if not code_string.strip():
        print("Warning: Empty code string")
        return False
    try:
        ast.parse(code_string)
        return True
    except SyntaxError as e:
        print(f"Syntax error in generated code: {str(e)}")
        return False

def improved_recursive_code_generation(task, depth=0, max_depth=3):
    print(f"Depth {depth}: Processing task: {task}")

    if depth >= max_depth:
        return "Max recursion depth reached. Task too complex."

    simplified_task = reframe_prompt(task)
    generated_code = executor(simplified_task)

    if is_valid_python(generated_code):
        print(f"Depth {depth}: Valid code generated")
        return generated_code

    print(f"Depth {depth}: Invalid code. Decomposing task")
    subtasks = improved_planner(simplified_task)

    subtask_results = []
    for subtask in subtasks:
        subtask_code = improved_recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    combined_code = "\n\n".join(subtask_results)
    return combined_code

# Test the recursive pipeline
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = improved_recursive_code_generation(main_task)

print("\nFinal Generated Code:")
print("=" * 40)
print(final_code)
print("=" * 40)

# Save the generated code to a file
with open("generated_code.py", "w") as f:
    f.write(final_code)
print("Generated code has been saved to 'generated_code.py'")

# Test the executor separately
test_prompt = "Write a Python function to calculate the factorial of a number."
test_result = executor(test_prompt)
print("\nTest Executor Result:")
print("=" * 40)
print(test_result)
print("=" * 40)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Depth 0: Processing task: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.
Executing prompt: Simplify and implement: Create a Python class for ...
Generated code length: 795
Syntax error in generated code: unterminated string literal (detected at line 29) (<unknown>, line 29)
Depth 0: Invalid code. Decomposing task
Executing prompt: List subtasks for: Simplify and implement: Create ...
Generated code length: 646
Depth 1: Processing task: 1. Simplify the problem to make it easier to implement.
Executing prompt: Simplify and implement: 1. Simplify the problem to...
Generated code length: 726
Syntax error in generated code: illegal target for annotation (<unknown>, line 1)
Depth 1: Invalid code. Decomposing task
Executing prompt: List subtasks for: Simplify and implement: 1. Simp...
Generated code length: 671
Depth 2: Processing task: 2. Implement the solution to the simplified problem.
Executing prompt: Simplify and implement:

In [ ]:
# Save the generated code to a file
with open("generated_code_test.py", "w") as f:
    f.write(test_result)
print("Generated code has been saved to 'generated_code_test.py'")

Generated code has been saved to 'generated_code_test.py'


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, GenerationConfig
import torch
import ast
from sklearn.cluster import KMeans

# Load CodeLlama model and tokenizer
codellama_name = "codellama/CodeLlama-7b-hf"
codellama_tokenizer = AutoTokenizer.from_pretrained(codellama_name)
codellama_model = AutoModelForCausalLM.from_pretrained(codellama_name, torch_dtype=torch.float16, device_map="auto")

# Set pad_token_id explicitly for CodeLlama
codellama_tokenizer.pad_token_id = codellama_tokenizer.eos_token_id

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Create a generation config for CodeLlama
generation_config = GenerationConfig(
    max_length=200,
    pad_token_id=codellama_tokenizer.pad_token_id,
    eos_token_id=codellama_tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
)

def reframe_prompt(prompt):
    inputs = bert_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    simplified_prompt = f"Simplify and implement: {prompt}"
    print(f"Reframed prompt: {simplified_prompt}")
    return simplified_prompt

def improved_planner(task):
    subtask_prompt = f"List subtasks for: {task}\nProvide the subtasks as a numbered list."
    subtask_text = executor(subtask_prompt)
    potential_subtasks = [line.strip() for line in subtask_text.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]

    if len(potential_subtasks) > 1:
        inputs = bert_tokenizer(potential_subtasks, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)

        n_clusters = min(len(potential_subtasks), 3)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(embeddings)

        subtasks = []
        for i in range(n_clusters):
            cluster_subtasks = [s for s, l in zip(potential_subtasks, kmeans.labels_) if l == i]
            if cluster_subtasks:
                subtasks.append(cluster_subtasks[0])
    else:
        subtasks = potential_subtasks

    print(f"Generated subtasks: {subtasks}")
    return subtasks

def executor(prompt):
    print(f"Executing prompt: {prompt}")
    inputs = codellama_tokenizer(prompt, return_tensors="pt").to(codellama_model.device)
    try:
        outputs = codellama_model.generate(**inputs, generation_config=generation_config)
        generated_code = codellama_tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Generated code length: {len(generated_code)}")
        print("Generated code:")
        print("=" * 40)
        print(generated_code)
        print("=" * 40)
        if len(generated_code) == 0:
            print("Warning: Generated code is empty")
        return generated_code
    except Exception as e:
        print(f"Error in code generation: {str(e)}")
        return ""

def is_valid_python(code_string):
    if not code_string.strip():
        print("Warning: Empty code string")
        return False
    try:
        ast.parse(code_string)
        return True
    except SyntaxError as e:
        print(f"Syntax error in generated code: {str(e)}")
        return False

def improved_recursive_code_generation(task, depth=0, max_depth=3):
    print(f"\nDepth {depth}: Processing task: {task}")

    if depth >= max_depth:
        print(f"Max recursion depth reached. Task too complex: {task}")
        return "Max recursion depth reached. Task too complex."

    simplified_task = reframe_prompt(task)
    generated_code = executor(simplified_task)

    if is_valid_python(generated_code):
        print(f"Depth {depth}: Valid code generated")
        return generated_code

    print(f"Depth {depth}: Invalid code. Decomposing task")
    subtasks = improved_planner(simplified_task)

    subtask_results = []
    for subtask in subtasks:
        subtask_code = improved_recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    combined_code = "\n\n".join(subtask_results)
    print(f"Depth {depth}: Combined code:")
    print("=" * 40)
    print(combined_code)
    print("=" * 40)
    return combined_code

# Test the recursive pipeline
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = improved_recursive_code_generation(main_task)

print("\nFinal Generated Code:")
print("=" * 40)
print(final_code)
print("=" * 40)

# Save the generated code to a file
with open("generated_code.py", "w") as f:
    f.write(final_code)
print("Generated code has been saved to 'generated_code.py'")

# Test the executor separately
test_prompt = "Write a Python function to calculate the factorial of a number."
test_result = executor(test_prompt)
print("\nTest Executor Result:")
print("=" * 40)
print(test_result)
print("=" * 40)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Depth 0: Processing task: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.
Reframed prompt: Simplify and implement: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.
Executing prompt: Simplify and implement: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.
Generated code length: 826
Generated code:
Simplify and implement: Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.

1. Simple Calculator

   Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods.

   - The calculator should have a property for the result.
   - The calculator should have properties for the current number, the previous number, and the current operator (+, -, *, /).
   - The calculator should have methods for inputting a

In [1]:
import logging
import os
import uuid
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, GenerationConfig
import torch
import ast
from sklearn.cluster import KMeans
import shutil

# Set up logging
logging.basicConfig(filename='code_generation.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Create a directory to store generated code files
if os.path.exists('generated_codes'):
    shutil.rmtree('generated_codes')
os.makedirs('generated_codes', exist_ok=True)

# Load CodeLlama model and tokenizer
codellama_name = "codellama/CodeLlama-7b-hf"
codellama_tokenizer = AutoTokenizer.from_pretrained(codellama_name)
codellama_model = AutoModelForCausalLM.from_pretrained(codellama_name, torch_dtype=torch.float16, device_map="auto")

# Set pad_token_id explicitly for CodeLlama
codellama_tokenizer.pad_token_id = codellama_tokenizer.eos_token_id

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Create a generation config for CodeLlama
generation_config = GenerationConfig(
    max_length=250,
    pad_token_id=codellama_tokenizer.pad_token_id,
    eos_token_id=codellama_tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
)

def save_generated_code(code, prefix='code'):
    filename = f"generated_codes/{prefix}_{uuid.uuid4().hex[:8]}.py"
    with open(filename, 'w') as f:
        f.write(code)
    return filename

def reframe_prompt(prompt):
    inputs = bert_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    simplified_prompt = f"Simplify and implement: {prompt}"
    logging.info(f"Reframed prompt: {simplified_prompt}")
    return simplified_prompt

def improved_planner(task):
    subtask_prompt = f"List subtasks for: {task}\nProvide the subtasks as a numbered list."
    subtask_text = executor(subtask_prompt)
    potential_subtasks = [line.strip() for line in subtask_text.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]

    if len(potential_subtasks) > 1:
        inputs = bert_tokenizer(potential_subtasks, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)

        n_clusters = min(len(potential_subtasks), 3)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(embeddings)

        subtasks = []
        for i in range(n_clusters):
            cluster_subtasks = [s for s, l in zip(potential_subtasks, kmeans.labels_) if l == i]
            if cluster_subtasks:
                subtasks.append(cluster_subtasks[0])
    else:
        subtasks = potential_subtasks

    logging.info(f"Generated subtasks: {subtasks}")
    return subtasks

def executor(prompt):
    logging.info(f"Executing prompt: {prompt[:100]}...")  # Log only the first 100 characters of the prompt
    inputs = codellama_tokenizer(prompt, return_tensors="pt").to(codellama_model.device)
    try:
        outputs = codellama_model.generate(**inputs, generation_config=generation_config)
        generated_code = codellama_tokenizer.decode(outputs[0], skip_special_tokens=True)
        logging.info(f"Generated code length: {len(generated_code)}")
        filename = save_generated_code(generated_code)
        logging.info(f"Generated code saved to: {filename}")
        return generated_code
    except Exception as e:
        logging.error(f"Error in code generation: {str(e)}")
        return ""

def is_valid_python(code_string):
    if not code_string.strip():
        logging.warning("Warning: Empty code string")
        return False
    try:
        ast.parse(code_string)
        return True
    except SyntaxError as e:
        logging.error(f"Syntax error in generated code: {str(e)}")
        return False

def improved_recursive_code_generation(task, depth=0, max_depth=3):
    logging.info(f"\nDepth {depth}: Processing task: {task}")

    if depth >= max_depth:
        logging.info(f"Max recursion depth reached. Task too complex: {task}")
        return "Max recursion depth reached. Task too complex."

    simplified_task = reframe_prompt(task)
    generated_code = executor(simplified_task)

    if is_valid_python(generated_code):
        logging.info(f"Depth {depth}: Valid code generated")
        return generated_code

    logging.info(f"Depth {depth}: Invalid code. Decomposing task")
    subtasks = improved_planner(simplified_task)

    subtask_results = []
    for subtask in subtasks:
        subtask_code = improved_recursive_code_generation(subtask, depth + 1, max_depth)
        subtask_results.append(subtask_code)

    combined_code = "\n\n".join(subtask_results)
    filename = save_generated_code(combined_code, prefix='combined')
    logging.info(f"Depth {depth}: Combined code saved to: {filename}")
    return combined_code

# Test the recursive pipeline
main_task = "Create a Python class for a basic calculator with addition, subtraction, multiplication, and division methods."
final_code = improved_recursive_code_generation(main_task)

logging.info("\nFinal Generated Code:")
filename = save_generated_code(final_code, prefix='final')
logging.info(f"Final code saved to: {filename}")

# Test the executor separately
test_prompt = "Write a Python function to calculate the factorial of a number."
test_result = executor(test_prompt)
logging.info("\nTest Executor Result:")
filename = save_generated_code(test_result, prefix='test')
logging.info(f"Test result saved to: {filename}")

print("Code generation process completed. Check 'code_generation.log' for details and 'generated_codes' directory for generated code files.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
ERROR:root:Syntax error in generated code: invalid character '’' (U+2019) (<unknown>, line 15)


Code generation process completed. Check 'code_generation.log' for details and 'generated_codes' directory for generated code files.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def load_model():
    model_id = "codellama/CodeLlama-7b-Instruct-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float32,  # Use float32 for CPU
        low_cpu_mem_usage=True,     # Optimize for lower CPU memory usage
        device_map="auto",
    )
    return tokenizer, model

def decompose_prompt(prompt, tokenizer, model):
    chat = [
        {"role": "system", "content": "You are a helpful assistant that decomposes coding tasks into subtasks."},
        {"role": "user", "content": f"Decompose the following coding task into subtasks. Provide only a numbered list of subtasks without any additional text:\n\n{prompt}"}
    ]

    inputs = tokenizer.apply_chat_template(chat, return_tensors="pt")

    # Generate in smaller chunks to manage memory
    max_length = 2048
    chunk_size = 256
    generated = inputs

    with torch.no_grad():  # Disable gradient calculation for inference
        for _ in range(0, max_length, chunk_size):
            chunk = model.generate(
                generated,
                max_new_tokens=chunk_size,
                temperature=0.7,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
            generated = chunk

    response = tokenizer.decode(generated[0], skip_special_tokens=True)

    # Extract subtasks from the response
    subtasks = [line.strip() for line in response.split('\n') if line.strip().startswith(('1.', '2.', '3.'))]
    return response

# Load the model once
tokenizer, model = load_model()

# Example usage
prompt = "Calculates the average of the sums of absolute differences between each pair of consecutive numbers for all permutations of a given list. Each permutation is shuffled before calculating the differences. Args: - numbers (list): A list of numbers. Default is numbers from 1 to 10. The function should output with: float: The average of the sums of absolute differences for each shuffled permutation of the list. You should write self-contained code starting with: ``` import itertools from random import shuffle def task_func(numbers=list(range(1, 3))): ```"
subtasks = decompose_prompt(prompt, tokenizer, model)
print(subtasks)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


KeyboardInterrupt: 

In [3]:
from datasets import load_dataset
dataset = load_dataset("bigcode/bigcodebench", split="v0.1.0_hf[:10]")

for example in dataset:
    prompt = example["instruct_prompt"]
    print("Prompt:" + prompt)
    print("-" * 50)
    subtasks = decompose_prompt(prompt, tokenizer, model)
    print(subtasks)
    print("=" * 50)